In [56]:
import numpy as np
import numpy.fft as fft
import matplotlib.pyplot as plt
from scipy import signal
from IPython import display
from PIL import Image
import math

%matplotlib inline

In [57]:
# number of points
n = 3

# range of points
r = 50

# number of images
m = 3

In [58]:
np.random.seed(805764)

lst = list(range(0, r + 1))

# psf
h = np.sort(np.hstack((np.random.choice(lst, (n, 1)), np.random.choice(lst, (n, 1)))), axis=0)

# each image
v1 = np.sort(np.hstack((np.random.choice(lst, (n, 1)), np.random.choice(lst, (n, 1)))), axis=0)
v2 = np.sort(np.hstack((np.random.choice(lst, (n, 1)), np.random.choice(lst, (n, 1)))), axis=0)
v3 = np.sort(np.hstack((np.random.choice(lst, (n, 1)), np.random.choice(lst, (n, 1)))), axis=0)
              

In [63]:
# calculated convolutions
b1 = np.empty((2, n, n))
b2 = np.empty((2, n, n))
b3 = np.empty((2, n, n))

# Each x, y dimension
for i in range(2):
    # Each point of (x, y) in h
    for j in range(n):
        # Each point of (x, y) in v
        for k in range(n):
            b1[i, j, k] = h[j, i] + v1[k, i]
            b2[i, j, k] = h[j, i] + v2[k, i]
            b3[i, j, k] = h[j, i] + v3[k, i]
            
# each b is a (x, y) list of points of where it's 1
b1 = np.hstack((np.reshape(b1[0], (b1.shape[1] * b1.shape[2], 1)), np.reshape(b1[1], (b1.shape[1] * b1.shape[2], 1))))
b2 = np.hstack((np.reshape(b2[0], (b2.shape[1] * b2.shape[2], 1)), np.reshape(b2[1], (b2.shape[1] * b2.shape[2], 1))))
b3 = np.hstack((np.reshape(b3[0], (b3.shape[1] * b3.shape[2], 1)), np.reshape(b3[1], (b3.shape[1] * b3.shape[2], 1))))


In [83]:
# each row of A represents an equation like y[i] + x1[j].
# each column of A represents a measurement x1, x2, x3
A = np.array([
     [1 ,0 ,0 ,1 ,0 ,0],
     [1 ,0 ,0 ,0 ,1 ,0],
     [1 ,0 ,0 ,0 ,0 ,1],
     [0 ,1 ,0 ,1 ,0 ,0],
     [0 ,1 ,0 ,0 ,1 ,0],
     [0 ,1 ,0 ,0 ,0 ,1],
     [0 ,0 ,1 ,1 ,0 ,0],
     [0 ,0 ,1 ,0 ,1 ,0],
     [0 ,0 ,1 ,0 ,0 ,1],
     [0 ,0 ,0 ,1 ,0 ,0]
])

vec = np.random.permutation(9)
rness = 3

In [84]:
b1 = A.dot(np.vstack((v1, h)))
b1 = b1[:b1.shape[0] - 1,]
besterror = np.linalg.norm(np.vstack((b1[vec], np.array([[0, 0]]))) - A.dot(np.linalg.pinv(A)).dot(np.vstack((b1[vec], np.array([[0, 0]])))))
print('initial: ' + str(besterror))
for iteration in range(20):
    print('iteration ' + str(iteration))
    order = np.random.permutation(b1.shape[0])
    for i in range(b1.shape[0]):
        vectemp = vec
        swapdistance = np.round(np.random.randn(1)[0] * rness)
        swapind = int(min(max(order[i] + swapdistance, 0), b1.shape[0] - 1))
        vec[order[i]] = vectemp[swapind]
        vec[swapind] = vectemp[order[i]]
        error = np.linalg.norm(np.vstack((b1[vec], np.array([[0, 0]]))) - A.dot(np.linalg.pinv(A)).dot(np.vstack((b1[vec], np.array([[0, 0]])))))
        if error < besterror:
            besterror = error
            print('improved: ' + str(besterror))
        else:
            vec = vectemp
    if besterror < 1e-1:
        break
print('took ' + str(iteration) + ' iterations')

initial: 36.823905279043935
iteration 0
improved: 34.227993741315835
improved: 27.002057534771517
improved: 19.316084259267225
improved: 15.67021236472421
improved: 7.211102550927983
iteration 1
improved: 7.211102550927981
improved: 7.211102550927978
iteration 2
iteration 3
improved: 8.788147155587888e-14
took 3 iterations
